# RL in Continuos Spaces

## Fundamental concepts.

Reinforcement Learning problems are tipically framed as Markov Decission Processes (MDP). These MDPs consist of a set of states 'S' and actions 'A' along with probabilities 'P', rewards'R' and a discount factor gamma ($\gamma$). 'P' captures how frequently different transitions and rewards occur, often modeled as a single joint probability where the state and reward at any time step $t +1$ depend only on the state and action taken at the previous step $t$ - this characteristic of certain environments is known as the Markov property).<br>
In an MDP there are two values in which we are normally interested in: <u>State values $V(s)$</u> - which we try to estimate or predict - and the <u>value of an action taken in a certain state Q(s, a)</u> - which can help us decide what action to take.

These two mappings (of functions) are very much interrelated and help us find an optimal policy for our problem $\pi^*$ - that maximizes the total reward received.

MDPs are probabilistic in nature; we can't predict with complete certainty what future rewards we will get and for how long. That is why we tipically aim for a <u>total expected reward</u> - the discount factor gamma (i.e.$\gamma$) comes into play here; it is used to assign a lower weight to future rewards when computing state and action values.

<h3> RL - The Framework</h3><br>
Markov Decision Process (MDP):

$\text{(}S, A, P, R, \gamma\text{)}$

 * State Transition and Reward Model: $\mathbb{P}(S_{t+1}, R_{t+1} \mid S_t, A_t)$
 * State Value Function: $V(S)$
 * Action Value Function: $Q(S,A)$

Goal is to find the optimal policy $\pi^*$ that maximizes the total expected reward

<h3> RL - Algorithms</h3><br>
Model-BasedLearning (Dynamic Programming)

 * Policy Iteration
 * Value Iteration

Model-Free Learning

 * Monte Carlo Methods
 * Temporal-Difference Learning

## Discrete vs. Continuos Spaces

Discrete spaces makes things simpler when dealing with RL problems. They allow us to represent any function of states and actions as dictionaries and/or look-up tables.<br>
Discrete spaces are in fact, critical to a number of RL algorithms. In some algorithms, during the value iteration, the internal for-loop goes over each state, one by one to update the corresponding value estimate $V(S)$ of each of these states. This would be impossible to do with continuour state spaces! - The internal loop would never end going through the states and updating their value estimates. This can also be a problem with discrete state spaces that contains a huge number of states; using these algorithms can quickly become infeasible.

For example, model-free methods like Q-Learning assume discrete state spaces. Remember that in this algorithm the maxQ is computed over all possible actions from state $S'$. This computation is possible when there is a finit set of actions. In contrast, when having continuous spaces (i.e. state and action spaces) this step becomes a full-blown optimization problem. This is the case as well when having discrete spaces with enormous number of states and/or actions.

A continuous space is not restricted to a set of distinct values like integers - they can take a range of values, typically real numbers. This means that quantities like state values that could be depicted as a bar chart in discrete cases (one bar for each state), will be now thought of as a density plot over a desired range.

In real life however, problems cannot always be described with any given numberof states and actions.<br>
In nature, most actions that take place in a physical environment are continuous in nature. Thus, we need to modify our representations or algorithms (or both) to accomodate continuous spaces.<br>
The two main strategies are **Discretization** and **Function Aproximation**.

### Discretization

Discretization is basically converting a continuous space into a discrete one. For some environments (maybe not all) discretizing the state-space can work out very well. It enables us to use existing algorithms with little or no modification. Both state spaces and action spaces can be discretized. An example of discretization could be, to devide angles in whole degrees or in increments of 15, 30 ,45, 90, etc., degrees - depending on what is more appropriate for the problem in question.<br>
We can divide for example, a complete space (an area) using a grid. Let's imagine we have a cleaner robot that cleans (of course) this area. We can have obstacles in the area, like forniture, and the robot must avoid crashing into the fornitures. With a grid representation the roboto can avoid crasing with the fornitures by completely avoiding the grids which are occupied by the fornitures. *This is known as accupancy grid*.<br>
Depending on the problem, we decide how big or small we want or grids to be.<br>
We could for example divide up the grid into smaller cells where required. It would still be an approximation, but it'd allows us to allocate more of our state representation to where it matters and it's better than dividing the entire state into finer cells, which may increase the total number of states and in turn, the tiem needed to compute value functions.

In [this 'Discritizing' notebook](./notesMiniProjects/Discretization_Tiling/notebooks/Discretization.ipynb), it can be seen an example application of this technique.

### Tile Coding

If there is a previous knowledge about the state space, it can be manually designed an appropriate discretization scheme. But if such a discretization scheme is to work in an arbitrary environment, then it is needed a more generic method. One elegant approach for this is *tile coding*.<br>
In tile coding the undrelying state space is continuous and two dimensional and we can overlay multiple grids on top of the space, each slightly offset from each other. Any position $S$ in the space can be coarsely identified by the tiles that it activates. If for example, we assign a bit to each tile, then we can represent our new discretized state as a bit vector, with ones for the tiles that get activated and zeros elsewhere (this by itself is a very efficient representation). The interesnting here is how the state value function is computed using the scheme. Instead of storing a separate value for each state $V(s)$, it is defined in terms of the *bit vector for that state* and a *weight for each tile*. The tile coding algorith in turn updates these weights iteratively. This ensures nearby locations that share tiles also share some component of state value, effectively smoothing the learned value function.

This tile coding approach have some drawbacks - we have to manaully select the sizes, the offsets, number of tilings, etc., ahead of time.

![tileCodingAgo](./images/tileCodingAlgo.png)

But there is a more flexible approach: *adaptive tile coding*. In this approach one can start with a fairly large tiles, and divide each tile into two whenever it is appropriate. How to know when to split? Basically, whenever the agent is not learning much with the current representation. - when the value function isn't changing. The agent can be instructed to stop when it reaches some upper limeit on the nubmer of splits, or some max iterations have been performed. But, which tile to split? We need to look at which one is likely to have the greatest effect on the value function. For this, we need to keep track of subtiles and their projected weights. Then, we simly pick the tile with the greatest difference between subtile weights.

There are many other heuristics that can be used. The advantage of adaptive tile coding is that it dows not rely on a human to specify a discretization ahead of time. The resulting state space is appropriately partitioned based on its complexity.

### Coarse Coding

Coarse coding is like Tile coding but uses a sparser set of features to encode the state space. As an example, imagine dropping a bunch of circles on a 2D continuous state space. Take any state $s$ which is a position in the space (in the state space) and mark all the circles that it belongs to. Prepare a bit vector with a "1" for those circles and "0" for the rest. That's your spacrse coding representation of the state. This also extends to higher dimensions where circles become spheres and hyperspheres.

![CoarseCoding](./images/coarseCoding.png)

Some properties of Coarse Coding: using smaller circles results in less genralization across the space. That means that the learning algorithm has to work longer but it results in a greater effective resolution. Larger circles lead to more generalzation and in general smoother value function. If we use fewer large circles to cover the space, we reduce the representation but will loose some resolution.<br>
Also, it is not only about the size of the circles. We can also change them in other ways like making them taller or wider to get more resolution along one dimension versus the other.<br>
This technique generalizes to pretty much any shape.<br>
A natural extension to this idea is to use the distance from the center of each circle as a measurement of how active that feature is. This measure (response) can be made to fall off smoothly using a Gaussian or a bell-shaped curve centered on the circle, which is known as a *radial basis function*.
The inconvenient with this approach is that the resulting feature values will no longer be discrete - you will end up with another continuous state vector **but** in this approach the number of features can be drastically reduced. We'll cover radial basis functions a bit later.

A bit of Tile Coding in practice [here](./notesMiniProjects/Discretization_Tiling/notebooks/Tile_Coding.ipynb).

## Function Approximation

Discretizing state spaces (or action spaces) enables us to use existing learning algorithms with little or no modification. But there are some limitations when the underlying space is complicated. In some cases the number of discrete states needed can become very large - in such cases we lose the advantage of discretization. Apart from this, it would be expected that -for instance- the values of two positions that are near from each other in the space are kind of similar (or close from each other as well). Discretization doesn't always exploit this characteristic, failing to generalize well across the space.

We are after the true state value $V_\pi$, or after the true action value function $q_\pi$, which is typically smooth and continuous over the entire space. As it can be imagined, capturing this completely is practically infeasible except for some very simple problems. So, our best hope is *function approximation*. It is still an approximation because *we don't know what the true underlying function is*.

A general way to define such an approximation is to introduce a parameter vector $W$ that shapes the function. Our task then reduces to tweaking this parameter vector till we find the desired approximation.
$$\hat{v}(s, W) \approx v_\pi(s)$$
$$\hat{q}(s, a, W) \approx q_\pi(s, a)$$

The approximation function can map a state to its value or a state-action pair to the corresponding $q$ value. Another form is where we map from one state to a number of different q-values, one for each action all at once (this is especially usefull for q-learning - to be seen later).

![DifferentMappingsFunctApprox](./images/functionApproximation.png)

### Approximating a state value function

So, we need to find a box (or a way) that converts the state $s$ and parameter vector $W$ into a scalar value.<br>

![Feature Vector](./images/featureVector.png)


The first thing needed, is to ensure we have a vector representing the state (our state might already be a vector in which case we don't need to do anything). In general, we'll define a transformation that converts any given state $s$ into a feature vector $x(s)$. This also gives us more flexibility, since we don't have to operate on the raw state values - we can use any computed or derived features instead:


$\begin{align}
\hat{v}(s, W)\text{ = ?} &\text{     ; value function} \\
x(s) = \begin{bmatrix}
x_1(s) \\
x_2(s) \\
\vdots \\
x_n(s)
\end{bmatrix} &\text{     ; feature vector}
\end{align} $


We now have a feature vector $X(s)$, and a parameter vector $W$ and we want a scalar value of these two. To produce a scalar out of two vectors we do a simple *Dot product*.<br>
A Dot product is the same as computing a linear combination of features. Multiply each feature with the corresponding weight, and sum it up.<br>
This is *linear function approximation* - we are trying to approximate the underlying value function with a linear function.

$$\hat{v}\big(s, W\big) = X(s)^\top \cdot W$$

$\begin{align}
\hat{v}(s,W) &= \big(x_1(s) \cdots x_n(s)\big) \cdot \begin{bmatrix}w_1\\ \vdots \\ w_n\end{bmatrix}\\
&= x_1(s) \cdot w_1 + \cdots + x_n(s) \cdot w_n \\
&= \sum_{j=1}^n x_j(s) \cdot w_j
\end{align} $


Given a problem domain with continuous states s∈S=Rns∈S=Rn, we wish to find a way to represent the value function $v_{\pi}(s)$ (for prediction) or $q_{\pi}(s,a)$ (for control).

We can do this by choosing a parameterized function that approximates the true value function:

$\hat{v}(s,W) \approx v_{\pi}(s)$  
$\hat{q}(s,a,W) \approx q_{\pi}(s,a)$

Our goal then reduces to finding a set of parameters $W$ that yield an optimal value function. We can use the general reinforcement learning framework, with a Monte-Carlo or Temporal-Difference approach, and modify the update mechanism according to the chosen function.

### Feature Vectors

A common intermediate step is to compute a feature vector that is representative of the state: $x(s)$

### Linear Function Approximation

![Linear Function - Gradient Descent](./images/linearFunction-GradientDescent.png)

To find out our Linear Function approximation we want to estimate the parameter vector $W$. In order to do that, we need to remeber that a linear function is a simple sum over all features multiplied by their corresponding weights. After these weights were initialized randomly, it could be computed the value of a state $\hat{v}(s, W)$:

$$\hat{v}\big(s, W\big) = X(s)^\top \cdot W \longrightarrow \text{ value function}$$

The question is now, how to tweak $W$ to bring the approximation closer and closer to the true function?<br>
This sounds like a *numerical optimization* problem. Let's use gradient descent to find the optimal parameter verctor (if you remember correctly, gradient descent is nothing but a derivative).

Since $\hat{v}$ is a linear function, its derivative with respect to $W$ is simply the feature verctor $X(s)$.

$$\nabla_W \hat{v}(s, W) = X(s)$$

What we are trying to optimize: the difference between the true value function $v_\pi$ and the approximate value function $\hat{v}$ (we want to reduce or minimize this difference). Rememeber: we are not concerned with the sign of the error, that is, we simply want to drive the difference down toward zero, so we will write that difference as a square diffrence. *****To be more accurate, since reinforcement learning domains are typically stochastic, this is the expected squared error.

$$J(W) = \mathbb{E}_\pi\Big[\big(v_\pi(s) - X(s)^\top W\big)^2\Big] \longrightarrow \text{minimize this error}$$

This is the error (function) that we have to minimize. In order to do it (by means of gradient descent), we need to find the gradient or derivative of this function with respect to $W$. Using the chain of rule of differentiation, we get <u>-2</u> times the <u>value difference</u> times <u>the derivative of $\hat{v}$</u> - which has already been noted it was simply the feature vector $X(s)$.

$$\nabla_W J(W) = -2\big(v_\pi(s) - X(s)^\top W\big)X(s)$$

We removed the expectation operator here to focus on the error gradient indicated  by a simple state $s$, which we assume has been chosen stochastically.

If we are able to sample enough states, we can come close to the expected value.<br>
Pugging this error gradient equation into the general form of a gradient descent update rule, we get:

$$\Delta W = -\alpha\frac{1}{2}\nabla_W J(W) \longrightarrow \text {Update Rule}$$

Notice that in this update rule we are using - as usual - alpha as a step-size or learning parameter. Notice also that the "-1/2" is used only to cancel out the "-2" we got in the derivative.

**<u>And this is the basic formulation to be used to iteratively reduce the error for each sample state, until the approximate and true function are almost equal.</u>**

$$\Delta W = \alpha\big(v_\pi(s) - X(s)^\top W\big)X(s)$$

An intuitive explanation of how tradient descent optimizes the parameter vector:<br>

$\begin{align}
\Delta W &\longrightarrow \text{change weights} \\
\alpha &\longrightarrow \text{small step} \\
\big(v_\pi(s) - X(s)^\top W\big) &\longrightarrow \text{away from error} \\
X(s) &\longrightarrow \text{}direction
\end{align}$


To solve a model *<u>free control problem (to take actons in an unknown environment)</u>*, we need to approximate the action-value function. To do this, we can define a feature transformation that utilizes both state and action and then we can use the same gradient descent method as we have developed it for the state value function:

$$\hat{q}(s, a, W) = \text{?          ;          }X(s, a) == \begin{pmatrix}x_1(s, a) \\ x_2(s, a) \\ \vdots \\x_n(s, a) \end{pmatrix}$$

This is also called "<u>Action Value Approximation</u>".

### Computing all of the action-values at once.

#### Action <u>Vector</u> Approximation
Finally, let us look at the case where we wish the approximation function *<u>to compute all of the action-values at once</u>*.<br>
For this purpose, we can continue to use the same feature transformation as before, taking in both the state and action.<br>
But to generate the different action-values..... how to do so?<br>
Basically, we are trying to find $n$ different action-value functions - one for each action dimension. Now, *intuitively*, we can say that we know, these $n$ action-value functions are related to each other; that is why it makes sense to compute them together. To do this, we extend the weight vector and turn it into a matrix - each column of the matrix emulates a separate linear function, but the common features computed from the state and action keep these functions tied to each other.

If we have a problem domain with a continuous state space but with a discrete action space -which is very common- we can easily select the action with the maximum value. Without this sort of parallel processing we would have to parcel each action one by one and then find their maximum.

If the action space is continuous too, then this form allow us to output more than a single value at once. For example, if we were driving a car, we'd want to control both steering and throttle at the same time.

$$\text{Action Vector Approximation} \\\begin{align}\hat{q}(s, a, W) & = \\ & = \big(x_1(s, a) \cdots x_n(s ,a)\big) \cdot \begin{pmatrix}w_{11} & \cdots & w_{1m} \\ \vdots & \ddots & \vdots \\ w_{n1} & \cdots & w_{nm}\end{pmatrix}\\ & = \big(\hat{q}(s, a_1, W) \cdots \hat{q}(s, a_m, W)\big)\end{align}$$

![Action Vector Approximation](./images/ActionVectorApproximation.png)

Linear function approximation has some limitations. The primary limitation is that <u>we can only represent linear relationships between inputs and outputs</u>. With one dimiensional input, this is basically a line. In 2-D, it becomes a plane and so on. What to do if our underlying value function has a <u><i>non-linear shape</i></u>? In this case, a linear approximation can give a very **bad result.** That's when we need to start looking at non-linear functions.

## Kernel Functions

Kernel Functions are a simple extension to linear function approximation. They can help to capture <u><i>non-linear relationships</i></u>.<br>
Remember that the feature transformation is defined in a generic sense as something that takes a state or a state-action pair and produces a feature vector.
This feature transformation lies at the center of the *kernel functions* approach. <u>Each element of this vector</u> can be produced by a separate function which can be non-linear.

As an example, assume a state $S$ that is a single real number. We could then define the following:

$$X_1(s) = S \\ X_2(s) = S^2 \\ X_3(s) = S^3\\ \vdots$$

These are called *"Kernel Functions"* or *"Basis Functions"*. They transform the input state into a different space.

<u>Note that since our value function is still defined as a linear combination of these features, we can still use linear function approximation.</u>

This allows the value function to represent non-linear relationships between the input state and output value.

![kernelFunctions](./images/kernelFunctions.png)

Radial Basis functions are a very common form of kernel used for this purpose. Think of the current state $S$ as a location in the continuous state space depicted in the image below as a rectangle plane. Each Basis function is shown as a blob. The closer the state is to the center of the blob, the higher the response returned by the function. And the farther you go, the response falls off gradually with the radius. Hence the name Radial Basis function. Mathematically, this can be achieved by associating a Gaussian Kernel with each Basis function, with its mean serving as the center of the blob and standard devaition determining how sharply or smoothly the response falls off. For any given state, we can reduce the state representation to a verctor of responses from these Radial Basis functions. From that point anwards, we can use our same function approximation method.
![radialBasisFunction](./images/radialBasisFunction.png)

## Non-Linear Function Approximation

What it has been building so far is non-linear function approximation.<br>
We discussed how to capture non linear relationships between an input state and its outpu value by means of using arbitrary kernels like radial basis functions as the feature transformation. Using such a model, we still get that out output value is still linear with respect to the features. What if our underlying value function was truly non-linear with respect to a combination of these features values? To capture such complex relationships, let's pass our linear response obtained using the dot product thorough some nonlinear function $f$ (this is the basis of artificial neural networks remember?). Such a non-linear function is generally called an activation function and immensely increases the representational capacity of our approximation. We can iteratively update the parameters of any such function using gradient descent - learning rate alpha, times value difference, times the derivative of the function with respect to the weights.

![nonLinearFunctionApprox](./images/nonLinearFunctionApproximation.png)

## Summary

In this lesson:
Traditional reinforcement learning technics use a finite MDP to model an environment which limits us to environments with descrete state and action spaces.
In order to extend our learning algorithms to continuous spaces, we can do one of two things:

1. Discretize the state space:
    Discretization can be performed using a constant grid, tile coding or coarse coding. This inderectly leads to an approximation of the falue function.

2. Try to approximate the desired value functions
    This can be done by first defining a feature transformation and then computing a linear combination of those features.
    Using non-linear feature transforms like radial basis functions, allows us to use the same linear combination framework to capture some non-linear relationships.

In order to represent non-linear relationships across combinations of features, we can apply an activation function.
This sets us up to use deep neural networks for reinforcement learning.